In [ ]:
import numpy as np

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.all import (StartMeshcat, DiagramBuilder, MeshcatVisualizerCpp,
                        MakeRenderEngineVtk, RenderEngineVtkParams,
                        Simulator, RotationalInertia)
from pydrake.geometry import (
    Box,
    Cylinder
)

from pydrake.multibody.tree import (
    UnitInertia,
    SpatialInertia,
    RevoluteJoint,
    FixedOffsetFrame,
    WeldJoint
)

import time

from typing import Tuple, List
from featherstone_py.example_models import DoublePendulum
import matplotlib.pyplot as plt

Prepare Drake plant and all required entities

In [ ]:
model_name = "double_pendulum"

In [ ]:
def create_drake_plant(model: DoublePendulum, model_name: str):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-4)

    L0, L1, L2 = model.l0, model.l1, model.l2
    h, w, r = model.h, model.w, model.r
    m = model.m

    RGBA_Color = [0.5, 0.5, 0.5, 1]

    my_model_instance = plant.AddModelInstance(model_name)

    inertia_link_0 = SpatialInertia.MakeFromCentralInertia(
        m, [0, 0, L0/2], RotationalInertia(m*(3*r**2+L0**2)/12, m*(3*r**2+L0**2)/12, m*r**2/2))
    inertia_link_1 = SpatialInertia.MakeFromCentralInertia(
        m, [0, 0, L1/2], RotationalInertia(m*(w**2+L1**2)/12, m*(h**2+L1**2)/12, m*(h**2+w**2)/12))
    inertia_link_2 = SpatialInertia.MakeFromCentralInertia(
        m, [0, 0, L2/2], RotationalInertia(m*(w**2+L2**2)/12, m*(h**2+L2**2)/12, m*(h**2+w**2)/12))

    link_0 = plant.AddRigidBody(
        "link_0", my_model_instance, inertia_link_0)
    link_1 = plant.AddRigidBody(
        "link_1", my_model_instance, inertia_link_1)
    link_2 = plant.AddRigidBody(
        "link_2", my_model_instance, inertia_link_2)


    plant.RegisterVisualGeometry(
        link_0,
        RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0/2]),
        Cylinder(r, L0),
        "link_0",
        RGBA_Color)
    plant.RegisterVisualGeometry(
        link_1,
        RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L1/2]),
        Box(h, w, L1),
        "link_1",
        RGBA_Color)
    plant.RegisterVisualGeometry(
        link_2,
        RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L2/2]),
        Box(h, w, L2),
        "link_2",
        RGBA_Color)

    frame_on_link_0 = plant.AddFrame(FixedOffsetFrame(
        link_0,
        RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0])))

    frame_on_link_1 = plant.AddFrame(FixedOffsetFrame(
        link_1,
        RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L1])))

    plant.AddJoint(RevoluteJoint(
        name="joint_0_to_1", frame_on_parent=frame_on_link_0,
        frame_on_child=link_1.body_frame(), axis=[1, 0, 0]))

    plant.AddJoint(RevoluteJoint(
        name="joint_1_to_2", frame_on_parent=frame_on_link_1,
        frame_on_child=link_2.body_frame(), axis=[1, 0, 0]))

    plant.WeldFrames(
        frame_on_parent_P=plant.world_frame(),
        frame_on_child_C=link_0.body_frame(),
        X_PC=RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0]))

    plant.Finalize()

    return plant, scene_graph, builder

In [ ]:
double_pendulum = DoublePendulum()

In [ ]:
plant, scene_graph, builder = create_drake_plant(double_pendulum, model_name)

In [ ]:
renderer_name = "renderer"
scene_graph.AddRenderer(
    renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))
meshcat = StartMeshcat()

meshcat.Delete()
meshcat_vis = MeshcatVisualizerCpp.AddToBuilder(
    builder, scene_graph, meshcat)

In [ ]:
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

Run Drake and custom simulation

In [ ]:
def run_drake_simulation(
        start_position: np.ndarray,
        time_period: float,
        dt: float = 0.01
) -> Tuple[List[float], List[float]]:
    plant_context = plant.GetMyMutableContextFromRoot(diagram_context)

    plant.SetPositions(plant_context, plant.GetModelInstanceByName(model_name),
                       [start_position[0], start_position[1]])  # theta1, theta2

    simulator = Simulator(diagram, diagram_context)
    simulator.set_publish_every_time_step(True)
    simulator.set_target_realtime_rate(1)
    simulator.Initialize()
    diagram.Publish(diagram_context)

    elapsed = 0.

    theta1_trajectory = []
    theta2_trajectory = []

    while elapsed < time_period:
        elapsed += dt
        simulator.AdvanceTo(elapsed)
        state = plant.GetPositions(plant_context)
        theta1_trajectory.append(state[0])
        theta2_trajectory.append(state[1])

    return theta1_trajectory, theta2_trajectory

In [ ]:
from featherstone_py.forward_dynamics import InverseDynamicsUsingCRBA, InertiaMatrixMethod, InverseDynamicsUsingRNEA


In [ ]:
def run_custom_simulation(
        double_pendulum: DoublePendulum,
        forward_dynamics: InertiaMatrixMethod,
        start_position: np.ndarray,
        time_period: float,
        dt: float = 0.01
) -> Tuple[List[float], List[float]]:
    from featherstone_py.simulator import Simulator
    model = double_pendulum.to_featherstone_notation()
    # very high integration resolution is required to get results similar to Drake simulator
    # due to usage of first order integration scheme
    # therefore it's quite slow
    sim = Simulator(model, forward_dynamics=forward_dynamics, integration_resolution=100)

    q = start_position.copy()
    qd = np.array([0., 0.])
    g = np.array([0, 0, -9.81])
    tau = np.array([0., 0.])

    elapsed = 0.

    theta1_trajectory = []
    theta2_trajectory = []

    while elapsed <= time_period:
        elapsed += dt
        q, qd = sim.advance_to(q, qd, tau, elapsed)
        theta1_trajectory.append(q[0])
        theta2_trajectory.append(q[1])

    return theta1_trajectory, theta2_trajectory

In [ ]:
time_period = 10.
dt = 0.01
q = np.array([2., 0.2])

In [ ]:
drake_simulation_result = run_drake_simulation(q, time_period, dt)

In [ ]:
%%time
custom_simulation_result = run_custom_simulation(double_pendulum, InverseDynamicsUsingRNEA(), q, time_period, dt)

In [ ]:
%%time
custom_simulation_result = run_custom_simulation(double_pendulum, InverseDynamicsUsingCRBA(), q, time_period, dt)

Compare joint trajectories plot of simulations

In [ ]:
def plot_series(series1: List[float], series2: List[float], name1: str, name2: str) -> None:
    n = min(len(series1), len(series2))
    plt.plot(np.array(series1)[:n], label=name1, color='r')
    plt.plot(np.array(series2)[:n], label=name2, color='g')
    plt.show()

In [ ]:
plot_series(drake_simulation_result[0], custom_simulation_result[0], 'drake_trajectory', 'custom_sim_trajectory')

In [ ]:
plot_series(drake_simulation_result[1], custom_simulation_result[1], 'drake_trajectory', 'custom_sim_trajectory')

Animate simulation in Meshcat

In [ ]:
def animate_simulation_result(dt: float, simulation_result: Tuple[List[float], List[float]]):
    elapsed = 0.
    idx = 0

    theta1_trajectory, theta2_trajectory = simulation_result

    plant_context = plant.GetMyMutableContextFromRoot(diagram_context)

    start = time.time()
    while time.time() < start + 10 and idx < len(theta1_trajectory):
        elapsed += dt
        time.sleep(dt)
        plant.SetPositions(plant_context, plant.GetModelInstanceByName(model_name),
                           [theta1_trajectory[idx], theta2_trajectory[idx]])
        idx += 1
        diagram.Publish(diagram_context)

In [ ]:
animate_simulation_result(dt, drake_simulation_result)

In [ ]:
animate_simulation_result(dt, custom_simulation_result)